In [ ]:
pip install pyinform

In [ ]:
pip install nilearn

In [ ]:
import pyinform as pyin
from nilearn.input_data import NiftiLabelsMasker
from nilearn import connectome
from nilearn.input_data import NiftiLabelsMasker

def conectividad_mutual(parcelacion, datos, low_pass, high_pass, confound= "none"):
  harvard_masker = NiftiLabelsMasker(labels_img = parcelacion, standardize = True, detrend=True,low_pass = low_pass, high_pass = high_pass,t_r=2)
  if confound == "none":
    fmri_matrix = harvard_masker.fit_transform(datos)
  else:
    fmri_matrix = harvard_masker.fit_transform(datos,confounds = confound)
  matriz = np.absolute(fmri_matrix.transpose())
  conectividad = np.zeros((len(matriz),len(matriz)))
  for i in range(len(matriz)):
    for j in range(i+1,len(matriz)):
      conectividad[i][j] = pyin.mutual_info(matriz[i],matriz[j])
  np.fill_diagonal(conectividad,0)
  return conectividad

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
cd "G:/Shared drives/GENSyR/CNF (Complejidad y NeuroFísica)/Carpeta de trabajo/Tesis Juan y Diego/Datos Poblaciones/"

In [ ]:
def mutual_entropy(parcelacion, datos, confound, low_pass = 0.08, high_pass = 0.009):
  harvard_masker = NiftiLabelsMasker(labels_img = parcelacion, standardize = True, detrend=True,low_pass = low_pass, high_pass = high_pass,t_r=2)
  fmri_matrix = harvard_masker.fit_transform(datos,confounds = confound)
  matriz = fmri_matrix.transpose()
  matriz = matriz + np.min(matriz)
  conectividad = np.zeros((len(matriz),len(matriz)))
  for i in range(len(matriz)):
    for j in range(i+1,len(matriz)):
      conectividad[i][j] = pyin.mutual_info(matriz[i],matriz[j])
  np.fill_diagonal(conectividad,0)
  return conectividad



tarea = "Dexterity_Unadj"
parcelacion = "AAL"
sujetos = ["108525"]
sexo = "M"
poblacion = "+"

home = "G:/Shared drives/GENSyR/CNF (Complejidad y NeuroFísica)/Carpeta de trabajo/"
home_datos = "D:/Users/juanc/Datos HC/" + sexo + poblacion + tarea + "/"
home_resultados = home + "Tesis Juan y Diego/Datos Poblaciones/" + tarea
parcelacion_mascara = nilearn.image.load_img(home + "Datos Proyecto Conectoma Humano (HCP)/Parcelas/" + parcelacion + "/" + parcelacion + "_space-MNI152NLin6_res-1x1x1.nii.gz")


def conectividades_mutuas(sujetos,parcelacion, confound= "none", low_pass = 0.08, high_pass = 0.009):
  tareas = ["REST1","MOTOR","EMOTION","SOCIAL","GAMBLING","LANGUAGE","WM","RELATIONAL"]
  for sujeto in sujetos:
    for tarea in tareas:
      base_funcional = home_datos + sujeto + "/Funcional/" + tarea
      base_resultados = home_datos + sujeto 

      if tarea == "REST1": 
        niigz = nilearn.image.load_img( base_funcional + "/rfMRI_" + tarea + "_LR.nii.gz")
        confound = base_funcional + "/Movement_Regressors.txt"

      if tarea != "REST1":
        niigz = nilearn.image.load_img( base_funcional + "/tfMRI_" + tarea + "_LR.nii.gz")
        confound = base_funcional + "/Movement_Regressors.txt"

      conectividad = mutual_entropy(parcelacion, niigz,confound)
      conectividad = conectividad + conectividad.transpose()
      np.save(base_resultados + "/" tarea + "_ME_" + sujeto + ".npy", conectividad)
      print("lista la conectividad de" + tarea + "para el sujeto" + sujeto)

